<a href="https://colab.research.google.com/github/AbdullahAZaidi/Stereo_Calibration/blob/main/StereoCal_forPrimax_GOLDEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from io import StringIO
from pathlib import Path
import sys
from typing import NamedTuple, Any
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import h5py
import glob
import numpy as np
import cv2
import os
import time
import math

from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_excel('/content/drive/Shareddrives/External_Primax/Primax/20210507 26 pairs/Intrinsic_parameters_26 pairs_20210507.xlsx', dtype={'Unit SN':'string', 'Pair SN':'string'} )
# df_output = pd.DataFrame()
# df.append(pandas.Series(), ignore_index=True)

translation_x_min = 390
translation_x_max = 410

translation_y_min = -10
translation_y_max = 10

translation_z_min = -10
translation_z_max = 10

rotation_min = -0.0087
rotation_max = 0.0087

def load_camera_matrix(Camera):
  Fx = df.loc[Camera, ['Fx'][0]]
  Fy = df.loc[Camera, ['Fy'][0]]
  Cx = df.loc[Camera, ['Cx'][0]]
  Cy = df.loc[Camera, ['Cy'][0]]
  
  return np.array([[ Fx,  0.,  Cx ],
                  [ 0.,  Fy,  Cy ],
                  [ 0.,  0.,  1 ]])


def load_dist_coeff(Camera):
  K1 = df.loc[Camera, ['K1'][0]]
  K2 = df.loc[Camera, ['K2'][0]]
  P1 = df.loc[Camera, ['P1'][0]]
  P2 = df.loc[Camera, ['P2'][0]]
  K3 = df.loc[Camera, ['K3'][0]]

  return np.array ([K1, K2, P1, P2, K3])


def translation_x_highlight(val):
    color = 'green' if translation_x_min <val < translation_x_max else 'red'
    return 'color: %s' % color

def translation_y_highlight(val):
    color = 'green' if translation_y_min <val < translation_y_max else 'red'
    return 'color: %s' % color

def translation_z_highlight(val):
    color = 'green' if translation_z_min <val < translation_z_max else 'red'
    return 'color: %s' % color

def rotation_highlight(val):
    color = 'green' if rotation_min <val < rotation_max else 'red'
    return 'color: %s' % color


# df_output = pd.DataFrame()
def write_trans_rot_errors(output, pair, baseline):

  vals = {'translation_x': [output[0]],
          'translation_y': [output[1]],
          'translation_z': [output[2]],
          'rotation_x': [output[3]],
          'rotation_y': [output[4]],
          'rotation_z': [output[5]],
          'pair SN': pair,
          'baseline': baseline
        }

  return vals
  # df_output = pd.DataFrame()
  # df_output_new = df_output.append(pd.DataFrame(vals, columns = ['pair SN','translation_x', 'translation_y', 'translation_z','rotation_x','rotation_y','rotation_z']) )
  # # new_df = pd.concat([df, df], axis=0)
  # return df_output_new


def write_output(sheet, df_output):
  # df.append(df)
  df_output.reset_index(drop=True)
  df_output.style.\
    applymap(translation_x_highlight,subset=['translation_x']).\
    applymap(translation_y_highlight,subset=['translation_y']).\
    applymap(translation_z_highlight,subset=['translation_z']).\
    applymap(rotation_highlight,subset=['rotation_x','rotation_y','rotation_z']).\
    to_excel(sheet, index = True, engine='openpyxl')

def intrinsics_vals(output):

  vals = {'Fx': [output.cam_mat[0,0]],
          'Fy': [output.cam_mat[1,1]],
          'Cx': [output.cam_mat[0,2]],
          'Cy': [output.cam_mat[1,2]],
          'K1': [output.dist_coeffs[0][0]],
          'K2': [output.dist_coeffs[0][1]],
          'K3': [output.dist_coeffs[0][4]],
          'P1': [output.dist_coeffs[0][2]],
          'P2': [output.dist_coeffs[0][3]],
          'K4': 0,
          'K5': 0,
          'K6': 0,
          'Projection Error' : [output.rms_reprojection_err]
          
        }

  return vals

def write_intrinsics(sheet, left, right):
  df_left = pd.DataFrame(intrinsics_vals(left), columns = ['pairs','Fx', 'Fy', 'Cx','Cy','K1','K2','K3','K4','K5','K6', 'P1','P2', 'Projection Error' ])
  df_right = pd.DataFrame(intrinsics_vals(right),columns = ['pairs','Fx', 'Fy', 'Cx','Cy','K1','K2','K3','K4','K5','K6', 'P1','P2', 'Projection Error' ])

  new_df = pd.concat([df_left, df_right], axis=0)
  # df_left.append(df_right)
  new_df.to_excel(sheet, index = False, engine='openpyxl')
  # df_right.to_excel(sheet, index = False, engine='openpyxl', startrow = 1)
  # df.to_excel(sheet, index = False, header=True)
                  

def _find_checkerboard_corners(tgt_size, imfile_path: Path) -> Any:
    print(f'Loading {imfile_path} ...', file=sys.stderr)
    assert imfile_path.is_file(), f"{imfile_path} is not a valid file"
    im = cv2.imread(str(imfile_path))
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#     plt.imshow(im)

    print('  finding chessboard corners ...', file=sys.stderr)
    pattern_was_found, pixel_corners = cv2.findChessboardCorners(gray_im, tgt_size, flags=cv2.CALIB_CB_FAST_CHECK)
    assert pattern_was_found, f"No corners detected!"
    cv2.cornerSubPix(gray_im, pixel_corners, (11, 11), (-1, 1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

    if (pixel_corners[0] > pixel_corners[-1]).all():
        print(f'Flipping reversed checkboard detections for {imfile_path}', file=sys.stderr)
        pixel_corners = pixel_corners[::-1]  # Corners must be top-bottom

    print('  writing corner detection visualization ...', file=sys.stderr)
    cv2.drawChessboardCorners(im, tgt_size, pixel_corners, pattern_was_found)
    cv2.imwrite(f"plots/{imfile_path.name}", im)

    return pixel_corners, gray_im.shape[::-1]

def _find_checkerboard_corners2(tgt_size, imfile_path: Path) -> Any:
    print(f'Loading {imfile_path} ...', file=sys.stderr)
    assert imfile_path.is_file(), f"{imfile_path} is not a valid file"
    im = cv2.imread(str(imfile_path))
    im = im[0:im.shape[0]*5//9, 0:im.shape[1]]
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    plt.imshow(im)

    print('  finding chessboard corners ...', file=sys.stderr)
    pattern_was_found, pixel_corners = cv2.findChessboardCorners(gray_im, tgt_size, flags=cv2.CALIB_CB_FAST_CHECK)
    assert pattern_was_found, f"No corners detected!"
    # cv2.cornerSubPix(gray_im, pixel_corners, (11, 11), (-1, 1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

    if (pixel_corners[0] > pixel_corners[-1]).all():
        print(f'Flipping reversed checkboard detections for {imfile_path}', file=sys.stderr)
        pixel_corners = pixel_corners[::-1]  # Corners must be top-bottom

    print('  writing corner detection visualization ...', file=sys.stderr)
    cv2.drawChessboardCorners(im, tgt_size, pixel_corners, pattern_was_found)
    cv2.imwrite(f"plots/{imfile_path.name}", im)

    return pixel_corners, gray_im.shape[::-1]


def _load_correspondences_impl(tgt_size, img_file_paths, patch_size: float=20):
    known_target_points = np.zeros((tgt_size[0] * tgt_size[1], 3), np.float32)
    known_target_points[:,:2] = np.mgrid[0:tgt_size[0], 0:tgt_size[1]].T.reshape(-1,2) * patch_size

    correspondences = {}
    for file_path in img_file_paths:
        pixel_corners, im_dimensions = _find_checkerboard_corners(tgt_size, file_path)
        correspondences[file_path] = (known_target_points, pixel_corners, im_dimensions)
    return correspondences

def _load_correspondences_impl2(tgt_size, img_file_paths, patch_size: float=10):
    known_target_points = np.zeros((tgt_size[0] * tgt_size[1], 3), np.float32)
    known_target_points[:,:2] = np.mgrid[0:tgt_size[0], 0:tgt_size[1]].T.reshape(-1,2) * patch_size

    correspondences = {}
    for file_path in img_file_paths:
        pixel_corners, im_dimensions = _find_checkerboard_corners(tgt_size, file_path)
        correspondences[file_path] = (known_target_points, pixel_corners, im_dimensions)
    return correspondences

def load_correspondences_checkerboard(img_file_paths):
    return _load_correspondences_impl((19, 13), img_file_paths)


def load_correspondences_cross_chart(img_file_paths):
    return _load_correspondences_impl2((14, 7), img_file_paths)


def angle_axis_to_matrix(angle_axis: np.ndarray) -> np.ndarray:
    """ Convert a rotation from an angle-axis representation to its
    matrix representation.

    Params
    ------
    `angle_axis`: numpy.ndarray with shape (3,)
        This vector represents a rotation about an axis. Its magnitude
        represents the angle of rotation, in radians, and its (unit-)
        direction represents the axis of rotation. Hence a rotation of
        π/4 about the z-axis would be represented as:
        `np.array([0, 0, np.pi/4])`.

    Returns
    -------
    A unitary, orthonormal, 4x4 transformation matrix that represents the
    same rotation as `angle_axis`.
    """
    magnitude = np.linalg.norm(angle_axis)
    angle, x, y, z = magnitude, *(angle_axis / magnitude)
    K = np.array([ 0, -z,  y,
                   z,  0, -x,
                  -y,  x,  0 ]).reshape(3, 3)
    # Compute exponential map (exp(K)) using Rodriguez's formula
    R = np.eye(3) + (np.sin(angle) * K) + ((1 - np.cos(angle)) * (K @ K))

    matrix_4x4 = np.eye(4)
    matrix_4x4[:3, :3] = R
    return matrix_4x4


def translation_matrix(x: float, y: float, z: float):
    """ Return a 4x4 matrix representating the (x, y, z) translation
    """
    return np.array([[ 1.,  0.,  0.,  x  ],
                     [ 0.,  1.,  0.,  y  ],
                     [ 0.,  0.,  1.,  z  ],
                     [ 0.,  0.,  0.,  1. ]])


def rtvec_to_matrix(rvec, tvec) -> np.ndarray:
    return translation_matrix(*tvec) @ angle_axis_to_matrix(rvec)


def matrix_to_xyzrph(M: np.ndarray):
    tx = M[0, 3]
    ty = M[1, 3]
    tz = M[2, 3]
    rx = np.arctan2(M[2, 1],  M[2, 2])
    ry = np.arctan2(-M[2, 0],  np.sqrt(M[0, 0]*M[0, 0] + M[1, 0]*M[1, 0]))
    rz = np.arctan2(M[1, 0],  M[0, 0])
    return tx, ty, tz, rx, ry, rz

class CameraCalibrationResult(NamedTuple):
    rms_reprojection_err: Any
    cam_mat: Any
    dist_coeffs: Any
    cam_from_target_poses: Any


def calibrate_camera(correspondences):
    obj_points = [ o for o, _i, _s in correspondences.values() ]
    img_points = [ i for _o, i, _s in correspondences.values() ]
    im_shapes = [ s for _o, _i, s in correspondences.values() ]
    assert all( im_shapes[0] == s for s in im_shapes )
    rmse, cam_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, im_shapes[0], None, None)
    cam_from_target_poses = [ rtvec_to_matrix(r.squeeze(), t.squeeze()) for r, t in zip(rvecs, tvecs) ]
    return CameraCalibrationResult(rmse, cam_matrix, dist_coeffs, cam_from_target_poses)

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 400 else 'black'
    return 'color: %s' % color


# this_column = df['K1']
# print(len(this_column))
# print (this_column)


def main():
  df_output = pd.DataFrame()

  unit = df['Unit SN']
  pair = df['Pair SN']
  print (len(unit))


  for i in range(1,len(unit),2) :

    print (unit[i], pair[i])

    left_cam_mat = load_camera_matrix(i)
    right_cam_mat = load_camera_matrix(i+1)

    #load distortion coefficients
    left_dist_coeff = load_dist_coeff(i)
    right_dist_coeff = load_dist_coeff(i+1)

    print(left_cam_mat, right_dist_coeff)

    lobj_points, limg_points, _ = list(load_correspondences_cross_chart([ Path('/content/drive/Shareddrives/External_Primax/Primax/20210507 26 pairs/Stereo Images/BMP/PairSN_{}_unitSN_{}_left.bmp'.format(pair[i],unit[i])) ]).values())[0]
    robj_points, rimg_points, _ = list(load_correspondences_cross_chart([ Path('/content/drive/Shareddrives/External_Primax/Primax/20210507 26 pairs/Stereo Images/BMP/PairSN_{}_unitSN_{}_right.bmp'.format(pair[i],unit[i+1])) ]).values())[0]

    _, r, t = cv2.solvePnP(lobj_points, limg_points, left_cam_mat, left_dist_coeff)
    leftcam_from_chart = rtvec_to_matrix(r.squeeze(), t.squeeze())
    
    _, r, t = cv2.solvePnP(robj_points, rimg_points, right_cam_mat, right_dist_coeff)
    rightcam_from_chart = rtvec_to_matrix(r.squeeze(), t.squeeze())
  

    z = matrix_to_xyzrph(leftcam_from_chart @ np.linalg.inv(rightcam_from_chart))
    baseline = math.sqrt ((z[0])**2 + (z[1]**2) + (z[2]**2)) 
    p = write_trans_rot_errors(z, pair[i], baseline)
    df_output = df_output.append(pd.DataFrame(p, columns = ['pair SN','baseline','translation_x', 'translation_y', 'translation_z','rotation_x','rotation_y','rotation_z']) )
   
  sheet = '/content/gdrive/Shareddrives/External_Primax/Primax/Stereo Calibration/rough_work/Copy of Stereo_Alignment_Data.xlsx'

  df_output.to_excel(sheet, index = False, engine='openpyxl')



  time.sleep(10)


  df_new = pd.read_excel(sheet,dtype={'pair SN':'string'})
  df_new.style.\
    applymap(translation_x_highlight,subset=['translation_x','baseline']).\
    applymap(translation_y_highlight,subset=['translation_y']).\
    applymap(translation_z_highlight,subset=['translation_z']).\
    applymap(rotation_highlight,subset=['rotation_x','rotation_y','rotation_z']).\
    to_excel(sheet, index = True, engine='openpyxl')

    

if __name__ == '__main__':
    main()


KeyboardInterrupt: ignored

In [ ]:
sheet = '/content/gdrive/Shareddrives/External_Primax/Primax/Stereo Calibration/rough_work/Copy of Stereo_Alignment_Data.xlsx'

df = pd.read_excel(sheet,dtype={'pair SN':'string'})

df.style.\
  applymap(translation_x_highlight,subset=['translation_x']).\
  applymap(translation_y_highlight,subset=['translation_y']).\
  applymap(translation_z_highlight,subset=['translation_z']).\
  applymap(rotation_highlight,subset=['rotation_x','rotation_y','rotation_z']).\
  to_excel(sheet, index = True, engine='openpyxl')


    Unnamed: 0  Unnamed: 0.1  ...  rotation_y  rotation_z
0            0             0  ...    0.015316    0.002007
1            1             1  ...   -0.004579   -0.003490
2            2             2  ...   -0.037567   -0.006561
3            3             3  ...   -0.000976    0.004595
4            4             4  ...   -0.014773   -0.004907
5            5             5  ...   -0.021480   -0.004262
6            6             6  ...   -0.007323   -0.006130
7            7             7  ...   -0.030448   -0.005455
8            8             8  ...   -0.017832    0.001405
9            9             9  ...   -0.014053   -0.004327
10          10            10  ...   -0.015156   -0.003767
11          11            11  ...   -0.025361   -0.004500
12          12            12  ...   -0.017730   -0.003011
13          13            13  ...   -0.019568    0.001050

[14 rows x 13 columns]
